## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-03-02-13-15-52 +0000,bbc,Why did US and Israel attack Iran and how long...,https://www.bbc.com/news/articles/cx2dyz6p3weo...
1,2026-03-02-13-14-51 +0000,nyt,Live Updates: Mideast Conflict Widens Across M...,https://www.nytimes.com/live/2026/03/02/world/...
2,2026-03-02-13-14-00 +0000,wsj,"BlackRock, EQT-Led Consortium to Acquire AES f...",https://www.wsj.com/finance/investing/blackroc...
3,2026-03-02-13-12-55 +0000,nyt,Israel Strikes Beirut After Hezbollah Attacks,https://www.nytimes.com/2026/03/01/world/middl...
4,2026-03-02-13-12-30 +0000,bbc,Henry Zeffman: Starmer seeks to carve out dist...,https://www.bbc.com/news/articles/c0rj4nvw07do...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2586/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,iran,84
69,trump,27
21,strikes,20
5,war,20
51,middle,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
158,2026-03-02-01-17-20 +0000,latimes,Thousands rally in Westwood as U.S.-Iran war e...,https://www.latimes.com/california/story/2026-...,201
266,2026-03-01-15-31-00 +0000,nyt,Israel launches new military strikes as Iran r...,https://www.nytimes.com/live/2026/03/01/world/...,163
218,2026-03-01-20-15-06 +0000,nyt,Middle East Airspace Closes as U.S.-Israeli St...,https://www.nytimes.com/2026/02/28/world/middl...,161
138,2026-03-02-03-49-59 +0000,nyt,Iran Strikes U.S. Military Facilities in Dubai...,https://www.nytimes.com/2026/03/01/world/middl...,158
33,2026-03-02-12-19-08 +0000,nyt,"After Attack on Iran by the U.S. and Israel, S...",https://www.nytimes.com/2026/02/28/world/middl...,158


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
158,201,2026-03-02-01-17-20 +0000,latimes,Thousands rally in Westwood as U.S.-Iran war e...,https://www.latimes.com/california/story/2026-...
180,87,2026-03-01-22-57-46 +0000,nypost,Trump warns Iranian regime to surrender or fac...,https://nypost.com/2026/03/01/world-news/trump...
115,85,2026-03-02-07-45-00 +0000,wsj,Middle East Conflict Sparks Surge in Oil Prices,https://www.wsj.com/world/middle-east/middle-e...
266,79,2026-03-01-15-31-00 +0000,nyt,Israel launches new military strikes as Iran r...,https://www.nytimes.com/live/2026/03/01/world/...
79,51,2026-03-02-10-30-00 +0000,wsj,The Texas Republican Senate primary is getting...,https://www.wsj.com/politics/elections/texas-p...
209,40,2026-03-01-20-43-28 +0000,nypost,Trump reveals how long he expects Iran campaig...,https://nypost.com/2026/03/01/world-news/trump...
229,36,2026-03-01-19-32-00 +0000,wsj,Two Dead in Austin Bar Shooting; FBI Sees Poss...,https://www.wsj.com/us-news/two-dead-in-austin...
30,32,2026-03-02-12-27-21 +0000,nypost,Huge fire as one of the world’s largest oil re...,https://nypost.com/2026/03/02/world-news/drone...
270,32,2026-03-01-14-45-56 +0000,bbc,Months of planning behind US-Israeli mission t...,https://www.bbc.com/news/articles/c86y5540vnno...
73,31,2026-03-02-10-34-00 +0000,wsj,"Canada, India Strike $1.9 Billion Uranium Supp...",https://www.wsj.com/economy/canada-india-strik...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
